In [1]:
import dfBasics
import common
import encoder
import pfAdapt
import charts


Setup Complete


In [2]:
import pandas as pd
from pyspark.sql import functions

# Lib functions

# Main

In [3]:
sparkSession = dfBasics.getSparkSession()

In [4]:
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/stage1/v1/labelencoder/*/*')
#.limit(100000) 

In [5]:
missingSchemaids = [ 1480883705,  -404024316,  -183769575,  2031641327, -1576843338,
                     -660710506,  -208259125,  -650561809,   -39609584, -1115728007]

df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/stage1/flat_*/*') 
df = df.filter(~df['schemaid'].isin(*missingSchemaids)) \
       .withColumn('timestamp', col('timestamp').cast('long')) 

NameError: name 'col' is not defined

In [7]:
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/*/*') 

In [6]:
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/stage1/flat_' + str(cycle) + '/*') \
    .withColumn('timestamp', col('timestamp').cast('long')) 

NameError: name 'cycle' is not defined

# Functions

In [ ]:
def getMissingSchemaids(pf,hashes):
    a = np.array([])
    for hash in hashes:  
        if len(pf[pf['hashvalue'] == hash]) != len(pd.unique(pf[pf['hashvalue'] == hash]['timestamp'])):
            value = pd.unique(pf[pf['hashvalue'] == hash]['schemaid'])
            if (value in a) == False:
                a = np.append(a,value)
    return a.astype(int)

In [ ]:
def dfcolumnSeries(column):
    return df.select(column).dropDuplicates()
def dfcolumnCount(column):
    return dfcolumnSeries(column).count()

In [ ]:
from functools import reduce
import operator

def eval1(filteredrows):
    element = staticfeatureset.index[0]
    res = filteredrows[element].eq(staticfeatureset[element])

    for element in staticfeatureset.index:
        res = reduce(operator.and_,(res,filteredrows[element].eq(staticfeatureset[element])))
    return res

In [ ]:
# find out variable columns and type of value
from pyspark.sql.functions import col

selected = ['timestamp', 'value','year','month', 'day', 'hour', 'minute']

def getDFvalueWithHash(currenthash):
    pfm = df.filter(col("hashvalue") == currenthash) \
            .withColumn('value', col('value').cast('int'))\
            .select(selected).toPandas().sort_values('timestamp').reset_index() 
    return pfm

def getDFvalueWithHashSchema(schemaid,currenthash):
    pfm = df.filter(col("hashvalue") == currenthash) \
            .filter(col("schemaid") == schemaid) \
            .withColumn('value', col('value').cast('int'))\
            .select(selected).toPandas().sort_values('timestamp').reset_index() 
    return pfm

def getIgroupHashes(igroup):
        return df.filter(col("igroup") == igroup).select(["hashvalue"]).dropDuplicates().toPandas()['hashvalue']

In [ ]:
# remove columns which are variable to find related metrics
def getStaticFeatures(_hash):
    allcolumns = filteredrows[filteredrows['hashvalue'] == _hash].columns
    skipcolumns = ['timestamp', 'value',  'hashvalue', 'igroup', 'inode', 'year', 'month', 'day', 'hour', 'minute']
    featurecolumns = allcolumns[~allcolumns.isin(skipcolumns)]
    featurecolumns = pfAdapt.getVariableUniqueColums(filteredrows[featurecolumns])
    staticfeatureset = filteredrows[filteredrows['hashvalue'] == _hash][featurecolumns].iloc[0]
    return staticfeatureset

# Info

In [ ]:
row1 = df.agg({"timestamp": "min"}).collect()[0]
row2 = df.agg({"timestamp": "max"}).collect()[0]

dfminTimestamp = row1["min(timestamp)"]
dfmaxTimestamp = row2["max(timestamp)"]
dfrowCount     = df.count()
dfcolumnCount  = len(df.columns)

In [ ]:
print('min:',common.date(dfminTimestamp), 'max:', common.date(dfmaxTimestamp))
print('rows:',dfrowCount,'columns:',dfcolumnCount)

In [8]:
columns = df.limit(1).toPandas().columns
withoutColumns = ['timestamp', 'value','schemaid', 'hashvalue']
columns = columns[~columns.isin(withoutColumns)]

In [9]:
columns

Index(['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',
       'CSLABILLINGMONTH', 'CSENDERPROTOCOL', 'CSENDERENDPOINTID',
       'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',
       'CMESSAGETAT2', 'CSLADELIVERYTIME'],
      dtype='object')

In [ ]:
#
schemas = dfcolumnSeries('schemaid').toPandas()['schemaid']
igroups = dfcolumnSeries('igroup').toPandas()['igroup']

schemaidCount  = dfcolumnCount('schemaid')
hashvalueCount = dfcolumnCount('hashvalue')
igroupCount = dfcolumnCount('igroup')
print('schemaidCount: ',schemaidCount,'hashvalueCount: ',hashvalueCount,'igroupCount: ',igroupCount)

In [10]:
pfall = df.limit(100000).toPandas() 

In [11]:
pfall.head()

,CGLOBALMESSAGEID,CSTARTTIME,CENDTIME,CSTATUS,CSERVICE,CSLABILLINGMONTH,CSENDERPROTOCOL,CSENDERENDPOINTID,CINBOUNDSIZE,CRECEIVERPROTOCOL,CRECEIVERENDPOINTID,CSLATAT,CMESSAGETAT2,CSLADELIVERYTIME
0,4045e5b4-3752-11ea-82e0-1f38ac1e100b,1579063700270,1579063705212,SUCCESS,MaKo,None,AS2,695e8780-d831-11e8-93f7-7d35ac1b495c,196,MAIL,bad8a080-038a-11e9-b9de-0e92ac1b495c,4848,4942,1579063705118
1,36b3a412-3752-11ea-82e0-1f38ac1e100b,1579063684502,1579063691601,SUCCESS,MaKo,None,AS2,695e8780-d831-11e8-93f7-7d35ac1b495c,416,MAIL,bad8a080-038a-11e9-b9de-0e92ac1b495c,6992,7099,1579063691494
2,395a8128-3752-11ea-aee1-e350ac1e100c,1579063688747,1579063694939,SUCCESS,MaKo,None,AS2,695e8780-d831-11e8-93f7-7d35ac1b495c,416,MAIL,bad8a080-038a-11e9-b9de-0e92ac1b495c,6092,6192,1579063694839
3,7788aac2-3753-11ea-aee1-e350ac1e100c,1579064221547,1579064235098,SUCCESS,MaKo,None,MAIL,bad8a080-038a-11e9-b9de-0e92ac1b495c,14923,AS2,695e8780-d831-11e8-93f7-7d35ac1b495c,13378,13551,1579064234925
4,1b3add73-3752-11ea-aee1-e350ac1e100c,1579063638268,1579063680566,SUCCESS,MAKO_BE_ROUTER,None,AS2,00fed140-e0f1-11e8-be62-528eac1b495c,2492,AS2,4c508960-e28e-11e8-be62-528eac1b495c,3994,42298,1579063642262


In [ ]:
hashes = getIgroupHashes(6)

In [ ]:
_hash = hashes[10]
staticfeatureset = getStaticFeatures(_hash)

In [ ]:
relatedhashes = pd.unique(pfall[eval1(pfall)]['hashvalue'])
relatedhashes

In [ ]:
def getDataForRelatedHashes(relatedhashes):
    data2 = pd.DataFrame()
    start = True
    for hash_ in relatedhashes:
        c1 =  getDFvalueWithHash(int(hash_))
        if start:
            data2['date'] = c1['timestamp']
            start = False
        data2[str(hash_)] =  c1['value']
    return data2

In [ ]:
data = getDataForRelatedHashes(relatedhashes)
len(data)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#sns.lineplot(data=data2.iloc[:, :2])
sns.lineplot(data=data.iloc[:, 1:])

In [ ]:
data

In [ ]:
filteredrows = pfall[pfall['igroup'] == 6]

len(filteredrows)
hashes = pd.unique(filteredrows['hashvalue'])

In [ ]:
import numpy as np
getMissingSchemaids(pfall,hashes)

In [ ]:
data2 = pd.DataFrame()
#pfm = getDFvalueWithHash(int(relatedhashes[0]))
#
data2['date'] = dt['timestamp'].astype(str)
data2['value'] = dt['value']

In [ ]:
#hashes_d = dfcolumnSeries('hashvalue').toPandas()['hashvalue']
hashes_d = df.select(['hashvalue','schemaid']).dropDuplicates().toPandas()

In [ ]:
len(pd.unique(hashes_d['hashvalue']))


In [ ]:
#len(pd.unique(hashes_d))


dt = getDFvalueWithHash(int(hashes_d['schemaid'][0]),int(hashes_d['hashvalue'][0]))


In [ ]:
# check, if it is unique
len(dt['timestamp']), len(dt)

In [ ]:
len(pd.unique(pfall['hashvalue']))

In [ ]:
print(len(hashes))
len(pd.unique(pfall['hashvalue']))

In [ ]:
#getInfoForSchemas(filteredrows)
#len(pd.unique(filteredrows['__name__']))
#getEncoder('__name__').inverse_transform(pd.unique(filteredrows['__name__']))

In [ ]:
pfall[(pfall['schemaid'] == 1629035211) & (pfall['timestamp'] == 1587819506)]

In [ ]:
#getEncoder('igroup').inverse_transform(igroups)
for igroup in igroups:
    filteredrows = pfall[pfall['igroup'] == igroup]
    count = len(pd.unique(filteredrows['inode']))
    if count > 1:
        print(getEncoder('igroup').inverse_transform([igroup]),igroup,count)

In [ ]:
for hash_ in hashes:
    metric = filteredrows[filteredrows['hashvalue']==hash_]
    count = len(pd.unique(metric['value']))
    if count > 3:
        print(hash_,count)

In [ ]:
#continue with hash
#_hash=-2018833881
_hash=-283862276

In [ ]:
# !!!!!!!!!!! missing schemas
getMissingSchemaids(pfall)

In [ ]:
filteredrows[filteredrows['hashvalue'] == _hash]
filteredrows[filteredrows['hashvalue'] == _hash][featurecolumns].iloc[0]['__name__']

In [ ]:


#['schemaid','igroup']

#filteredrows[featurecolumns].head()
#getVariableUniqueColums(filteredrows[featurecolumns])

# there are still too many columns for _hash
#len(getVariableUniqueColums(filteredrows[featurecolumns]))
#len(featurecolumns)


# there are no variable columns left
#getVariableUniqueColums(filteredrows[filteredrows['hashvalue'] == _hash][featurecolumns])

# static feature values for metric _hash

#staticfeatureset[featurecolumns[1]]
#staticfeatureset


#featurecolumns

In [ ]:
currenthash     = int(relatedhashes[1])
currentfeatures = filteredrows[filteredrows['hashvalue'] == currenthash].iloc[0]
currentfeatures['igroup'],currentfeatures['inode'],currentfeatures['hashvalue'],getEncoder('__name__').inverse_transform([currentfeatures['__name__']])[0]

In [ ]:
pfm

In [ ]:
#data2 = data2.sort_values('date').reset_index()
c1

In [ ]:
data2

In [ ]:
ax = createLineplot(data2,16,10,1.4,title="",skip=500)
label(ax,10,90)

In [ ]:
data2.iloc[2]

In [ ]:
pfall.head()

In [ ]:
filteredrows[featurecolumns].head()

In [ ]:
# find out variable columns and type of value
#getVariableUniqueColums(pfm)
#selected = ['timestamp', 'value','year','month', 'day', 'hour', 'minute']

#which inode is used by metri hashvalue
used_inode = filteredrows[filteredrows['hashvalue'] == _hash]['inode'].iloc[0]

# other inodes for the igroup
inodes_igroup = pd.unique(filteredrows[filteredrows['igroup'] == 6]['inode'])
otherinodes = inodes_igroup[inodes_igroup!=used_inode]

#
pd.unique(filteredrows[filteredrows['igroup'] == 6]['__name__'])
pd.unique(filteredrows[(filteredrows['igroup'] == 6) & (filteredrows['__name__'] == 309)]['logical_system'])

In [ ]:
filteredrows[filteredrows['inode'] == inodes_igroup[0]]['hashvalue']


In [ ]:

        
metric = filteredrows[filteredrows['hashvalue']==_hash]
#metric = filteredrows[filteredrows['hashvalue']==hashes[0]]
#usedcolumns(metric)

keep = getVariableUniqueColums(metric)
#keepcolumns(dataall,keep)
#keep
len(metric),len(pd.unique(metric['value'])), pd.unique(metric['value'])
metric

In [ ]:
pfall1 = pfall[pfall['schemaid'] == schemas[0]]
mdcountsall = pfall1.groupby(['igroup','hashvalue'])
#['igroup'].count() 
#data2 = pd.DataFrame()
mdcountsall

In [ ]:
getEncoder('__name__').inverse_transform([pfall1.iloc[0]['__name__']])[0]

In [ ]:
def createData(pfall,month,outcome) :
    if outcome < 2 :
        mdcountsall = pfall[(pfall['month'] == month) & (pfall['outcome'] == outcome)].groupby(['year','month','day','hour'])['outcome'].count()
    else :
        if (month > 0) & (month < 13) :
            mdcountsall = pfall[(pfall['month'] == month)].groupby(['year','month','day','hour'])['outcome'].count()
        else :
            mdcountsall = pfall.groupby(['year','month','day','hour'])['outcome'].count()    
    a,b,c,d = get_ymdh(mdcountsall)
    data2 = pd.DataFrame()
    data2['date'] = get_ymdh_string(a,b,c,d)
    data2['outcome'] =  mdcountsall.reset_index()['outcome'].astype(int) 

    #for pivot table
    data2['hours'] =  d.astype(int) 
    data2['days']  =  c.astype(int) 
    piv = pd.pivot_table(data2, values="outcome",index=["hours"], columns=["days"], fill_value=0)
    return data2

In [ ]:
pfall.head()